In [434]:
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd

In [435]:
import PyPDF2 

In [436]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter#process_pdf
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
import string
from cStringIO import StringIO
import re
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import json

    # PDFMiner boilerplate

In [437]:
fp=file("G:/ml/imp/JavaBasics-notes.pdf", 'rb')

In [438]:
def extract_Text(files):
   
    l=[]
        # Extract text
    
    for page in PDFPage.get_pages(files):
        rsrcmgr = PDFResourceManager()
        sio = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, sio, codec=codec, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        interpreter.process_page(page)
        l.append(sio.getvalue())
        device.close()
        sio.close()
    
        # Get text from StringIO

   
        # Cleanup
    
    return l


In [439]:
list_pages=extract_Text(fp)

In [440]:
def convertLine(list_pages):
    spl =[]
    for x in list_pages:
        x = ' '.join(x.split())
        spl.append(x.lower())
    return spl
        

In [441]:
list_pages=convertLine(list_pages)

In [442]:
def remove_punc(st):
    st_spl=st.split()
    st_spl[:] = [value.translate(None, string.punctuation) for value in st_spl]
    st_spl[:] = [value for value in st_spl if not any(c.isdigit() for c in value)]
    st_spl[:] = [value.decode("ascii", errors="ignore").encode() for value in st_spl]
    
    return ' '.join(st_spl)

In [443]:
stop_words = set(stopwords.words('english'))

In [444]:

def filtered_sent(splitted_page):
    filtered_sentence = [w for w in splitted_page if not w in stop_words]
#     filtered_sentence =[]
#     for w in splitted_page:
#         if w not in stop_words:
#             filtered_sentence.append(w)
    return (filtered_sentence)

In [445]:
import nltk
import nltk

from nltk.stem import PorterStemmer
def stem_word(filtered_sentence):
    lemma = nltk.wordnet.WordNetLemmatizer()
    
    words=""
    for w in filtered_sentence:
        stemmer=lemma.lemmatize(w,pos="v")
        stemmer=lemma.lemmatize(stemmer)
        
        words+=stemmer+" "
    return words

In [446]:
def check_imp(prop):
    vectorizer = TfidfVectorizer(min_df=1,max_df=0.8,stop_words='english',analyzer="word")
    X = vectorizer.fit_transform(prop)
    idf =vectorizer.idf_
    m=dict(zip(vectorizer.get_feature_names(), idf))
    z = json.dumps(m)

    return (m)

In [447]:
list_pages=convertLine(list_pages)
preprocessed_list=[]
word_import={}
for x in list_pages:
#     page=split_pages(x)
    
    pag=remove_punc(x)
    
    filtered_sentence=filtered_sent(pag.split())
    
    stemmed_word = stem_word(filtered_sentence)
    preprocessed_list.append(stemmed_word)    
        
word_import=check_imp(preprocessed_list)

In [448]:
z= sorted(word_import.items(), key=lambda item: item[1],reverse =True)

In [449]:
top50 = z[:50]

In [454]:
l0=[]
l1=[]
for x in top50:
    l0.append(x[0])
    l1.append(x[1])



In [453]:
submission1 = pd.DataFrame({'Keywords': l0,'Weightage': l1})
submission1.to_csv('G:/ml/imp/submission1.csv')